In [ ]:
transform_id = 1

"""
************ SETUP - DON'T TOUCH **************
This section imports data from the configuration database
and should not need to be altered, molested or otherwise messed with. 
~~These are not the droids you are looking for~~
"""
from core.constants import BRANCH_NAME, ENV_BUCKET
from core.helpers.session_helper import SessionHelper
from core.models.configuration import Transformation
from dataclasses import dataclass
from core.dataset_contract import DatasetContract

db_transform = SessionHelper().session.query(Transformation).filter(Transformation.id == transform_id).one()

@dataclass
class DbTransform:
    id: int = db_transform.id ## the instance id of the transform in the config app
    name: str = db_transform.transformation_template.name ## the transform name in the config app
    state: str = db_transform.pipeline_state.pipeline_state_type.name ## the pipeline state, one of raw, ingest, master, enhance, enrich, metrics, dimensional
    branch:str = BRANCH_NAME ## the git branch for this execution 
    brand: str = db_transform.pipeline_state.pipeline.brand.name ## the pharma brand name
    pharmaceutical_company: str = db_transform.pipeline_state.pipeline.brand.pharmaceutical_company.name # the pharma company name
    publish_contract: DatasetContract = DatasetContract(branch=BRANCH_NAME,
                            state=db_transform.pipeline_state.pipeline_state_type.name,
                            parent=db_transform.pipeline_state.pipeline.brand.pharmaceutical_company.name,
                            child=db_transform.pipeline_state.pipeline.brand.name,
                            dataset=db_transform.transformation_template.name)


# CORE Cartridge Notebook::[Referral Date Enrichment]
![CORE Logo](assets/coreLogo.png) 

---
## Keep in Mind
Good Transforms Are...
- **singular in purpose:** good transforms do one and only one thing, and handle all known cases for that thing. 
- **repeatable:** transforms should be written in a way that they can be run against the same dataset an infinate number of times and get the same result every time. 
- **easy to read:** 99 times out of 100, readable, clear code that runs a little slower is more valuable than a mess that runs quickly. 
- **No 'magic numbers':** if a variable or function is not instantly obvious as to what it is or does, without context, maybe consider renaming it.

## Workflow - how to use this notebook to make science
#### Data Science
1. **Document your transform.** Fill out the _description_ cell below describing what it is this transform does; this will appear in the configuration application where Ops will create, configure and update pipelines. 
1. **Define your config object.** Fill out the _configuration_ cell below the commented-out guide to define the variables you want ops to set in the configuration application (these will populate here for every pipeline). 
2. **Build your transformation logic.** Use the transformation cell to do that magic that you do. 
![caution](assets/cautionTape.png)

### Configuration

In [ ]:
""" 
********* CONFIGURATION - PLEASE TOUCH ********* 
This section defines what you expect to get from the configuration application 
in a single "transform" object. Define the vars you need here, and comment inline to the right of them 
for all-in-one documentation. 
Engineering will build a production "transform" object for every pipeline that matches what you define here.

@@@ FORMAT OF THE DATA CLASS IS: @@@ 

<value_name>: <data_type> #<comment explaining what the value is to future us>

~~These ARE the droids you are looking for~~
"""

class Transform(DbTransform):
    '''
    YOUR properties go here!!
    Include your input dataset(s) here. Variable properties should be assigned to the exact name of
    the transformation as it appears in the Jupyter notebook filename.
    '''

In [ ]:
## Please place your value assignments for development here!!
## This cell will be turned off in production and Engineering will set to pull form the configuration application instead
trans_id = 'pmcTransactionId'
product = 'medication'
patient_id = 'pmcPatientId'
pharm = 'pharmName'
status_date = 'statusDate'
ref_date = 'refDate'
status =  'statusCode'
substatus =  'custStatusCode'
ic_status = 'integrichain_status'
ic_substatus = 'integrichain_sub_status'

### Description
What does this transformation do? be specific.

![what does your transform do](assets/what.gif)

This Transform provides the preliminary enrichment format. Takes patient data in from S3 and adjusts all possibly inaccurate reference dates.

### Transformation

In [ ]:
transform = Transform()

transform.name = 'DS_290'
transform.brand = 'ofev'
transform.state = 'enrich'
transform.pharmaceutical_company = 'bi'
transform.filesystem_path = 's3://ichain-dev/movahlx/bi/transactions'

transform.secret_name = 'bi'
transform.secret_type_of = 'database'

In [ ]:
import re

import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
from s3fs import S3FileSystem
#import mysql.connector as mysql

from core.secret import Secret

In [ ]:
def pandas_from_parquet_s3(file_path):
    
    s3 = S3FileSystem()
    df = (
        pq
        .ParquetDataset(file_path, filesystem=s3)
        .read_pandas()
        .to_pandas()
    )
    
    return df


def pandas_from_database(query, *args):
    
    connection = mysql.connect(
        host=secret.host, 
        user=secret.user, 
        passwd=secret.password, 
        port=secret.port, 
        charset='utf8'
    )
    
    query = query.format(*args)
    df = pd.read_sql(sql=query, con=connection)
    connection.close()
    
    return df


def pandas_to_parquet_s3(df, partition, file_path):
    
    s3 = S3FileSystem()
    table = pa.Table.from_pandas(df)
    
    pq.write_to_dataset(table, file_path, partition_cols=partition, filesystem=s3)

In [ ]:
df = pandas_from_parquet_s3(transform.filesystem_path)
df.head()

In [ ]:
def load_clean_file(cust_df, trans_id, product, patient_id, pharm, status_date, ref_date, status, substatus):
    
    # Extract and map relevant columns
    #cust_df = cust_input_df.loc[:,[trans_id,
    #                               patient,
    #                               patient_id,
    #                               pharm,
    #                               product,
    #                               status_date,
    #                               ref_date,
    #                               status,
    #                               substatus]]

    cust_df = cust_df.rename(columns={trans_id:'trans_id',
                                      patient_id:'patient_id',
                                      pharm:'pharm',
                                      product:'product',
                                      status_date:'status_date',
                                      ref_date:'ref_date',
                                      status:'status_code',
                                      substatus:'substatus_code'})
    
    # Convert dates to datetime format
    cust_df.status_date = pd.to_datetime(cust_df.status_date)
    cust_df.ref_date = pd.to_datetime(cust_df.ref_date)
    
    ## Extract brand from medication
    cust_df['product'] = cust_df['product'].apply(lambda x: x.split()[0].strip())
    
	## Convert status codes to uppercase
    cust_df.status_code = cust_df.status_code.str.upper()
    cust_df.substatus_code = cust_df.substatus_code.str.upper()
    
	## Fill missing referral_date with min(status_date)
	
    min_status_dates=cust_df.groupby(['patient_id','pharm','product'])['status_date'].min().reset_index().rename(columns={'status_date':'min_status_date'})
    
    cust_df = pd.merge(cust_df, min_status_dates, how='inner', on=['patient_id','pharm','product'])
    
    cust_df.loc[cust_df['ref_date'].isnull(), ['ref_date']] = cust_df['min_status_date']
    
    cust_df = cust_df.drop(['min_status_date'],axis=1).drop_duplicates()
    
    cust_df.sort_values(by=['patient_id', 'pharm', 'product', 'status_date'], ascending=[True, True, True, True], inplace=True)
    cust_df = cust_df.reset_index(drop=True)

    # Output clean customer file
    return cust_df

In [ ]:
#def referral_date_enrichment(df: pd.DataFrame, cols, statuses,
#                         table_columns,
#                         ref_date_enrichment_threshold):
def referral_date_enrichment(df: pd.DataFrame, table_columns, ref_date_enrichment_threshold):
    """
    """
#    status_date = cols.status_date
#    ref_date = cols.ref_date
#    patient = cols.patient
#    pharm = cols.pharm
#    product = cols.product
    min_date_df = df.groupby(['patient_id', 'pharm', 'product'])['status_date'].min().reset_index().rename(columns={'status_date': 'First_Status_Date'})
    df = pd.merge(df, min_date_df, how='inner', on=['patient_id', 'pharm', 'product'])
    min_ref_date_df = df.groupby(['patient_id', 'pharm', 'product'])['ref_date'].min().reset_index().rename(columns={'ref_date': 'First_Ref_Date'})
    df = pd.merge(df, min_ref_date_df, how='inner', on=['patient_id', 'pharm', 'product'])

    df['min_ref_day_diff'] = (df['First_Status_Date'] - df['First_Ref_Date']) / np.timedelta64(1, 'D')
    df['ref_day_diff'] = (df['First_Status_Date'] - df['ref_date']) / np.timedelta64(1, 'D')

    to_enrich_df = df[(df['min_ref_day_diff'] > ref_date_enrichment_threshold) & (df['ref_day_diff'] > ref_date_enrichment_threshold)]
    to_enrich_df['ref_date'] = to_enrich_df['First_Status_Date']
    #to_enrich_df = to_enrich_df[table_columns]
    enriched_ids = to_enrich_df['trans_id'].values.tolist()
    df = df[~(df['trans_id'].isin(enriched_ids))]
    df = df.append(to_enrich_df)
    #df = df[table_columns]
    return (df)

In [ ]:
def file_standard(cust_df, trans_id, product, patient_id, pharm, status_date, ref_date, status, substatus):

    cust_df = cust_df.rename(columns={'trans_id': trans_id,
                                      'patient_id': patient_id,
                                      'pharm': pharm,
                                      'product': product,
                                      'status_date': status_date,
                                      'ref_date': ref_date,
                                      'status_code': status,
                                      'substatus_code': substatus})
    return cust_df

In [ ]:
### Use the variables above to execute your transformation. the final output needs to be a variable named final_dataframe
df = load_clean_file(df, trans_id, product, patient_id, pharm, status_date, ref_date, status, substatus)
columns = list(df.columns)
final_dataframe = referral_date_enrichment(df, columns, 60)

In [ ]:
len(final_dataframe)

### Publish

In [ ]:
## that's it - just provide the final dataframe to the var final_dataframe and we take it from there
transform.publish_contract.publish(final_dataframe)